In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import pandas as pd

In [2]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# NASA Mars News

In [3]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')

In [4]:
# Retrieve the latest news title
news_title=soup.find_all('div', class_='content_title')[1].text
# Retrieve the latest news paragraph
news_p=soup.find_all('div', class_='article_teaser_body')[0].text
print(news_title)
news_p

NASA's Self-Driving Perseverance Mars Rover 'Takes the Wheel'


'The agency’s newest rover is trekking across the Martian landscape using a newly enhanced auto-navigation system.'

# JPL Mars Space Images - Featured Image

In [5]:
# Mars Image to be scraped
jpl_nasa_url = 'https://spaceimages-mars.com/'
images_url = 'https://spaceimages-mars.com/'

browser.visit(images_url)

html = browser.html

images_soup = bs(html, 'html.parser')

In [8]:
# Retrieve featured image link
relative_image_path = images_soup.find_all('img')[1]["src"]
featured_image_url = jpl_nasa_url + relative_image_path
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars2.jpg


# Mars Facts

In [10]:
# Scrape Mars facts
url='https://galaxyfacts-mars.com/'
tables=pd.read_html(url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [22]:
mars_fact=tables[0]
mars_fact=mars_fact.rename(columns={0:"Profile",1:"Mars",2:"Earth"},errors="raise")
mars_fact.set_index("Profile",inplace=True)
mars_fact

,Mars,Earth
Profile,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [23]:
fact_table=mars_fact.to_html()
fact_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Profile</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</

In [24]:
fact_table.replace('\n','')
print(fact_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
    <tr>
      <th>Profile</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Mars - Earth Comparison</th>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>Diameter:</th>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>Distance from Sun:</th>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>Length of Year:</th>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>Temperature:</th>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


# Mars Hemispheres

In [26]:
# Scrape Mars hemisphere title and image
url='https://marshemispheres.com/'
browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')

In [27]:
# Extract hemispheres item elements
mars_hems=soup.find('div',class_='collapsible results')
mars_item=mars_hems.find_all('div',class_='item')
hemisphere_image_urls=[]

In [38]:
# Loop through each hemisphere item
for item in mars_item:
    # Error handling
    try:
        # Extract title
        hem=item.find('div',class_='description')
        title=hem.h3.text
        
        # Extract image url
        hem_url=hem.a['href']
        browser.visit(url+hem_url)
        html=browser.html
        soup=bs(html,'html.parser')
        image_src=soup.find('li').a['href']
        
        if (title and image_src):
            
            # Print results
            print('-'*50)
            print(title)
            print(url+image_src)
            
        # Create dictionary for title and url
        hem_dict={
            'title':title,
            'image_url':image_src
        }
        
        hemisphere_image_urls.append(hem_dict)
        
    except Exception as e:
        print(e)

--------------------------------------------------
Cerberus Hemisphere Enhanced
https://marshemispheres.com/images/full.jpg
--------------------------------------------------
Schiaparelli Hemisphere Enhanced
https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg
--------------------------------------------------
Syrtis Major Hemisphere Enhanced
https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg
--------------------------------------------------
Valles Marineris Hemisphere Enhanced
https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg


In [39]:
hemisphere_image_urls

[{'title': 'Valles Marineris Hemisphere', 'img_url': '...'},
 {'title': 'Cerberus Hemisphere', 'img_url': '...'},
 {'title': 'Schiaparelli Hemisphere', 'img_url': '...'},
 {'title': 'Syrtis Major Hemisphere', 'img_url': '...'},
 {'title': 'Cerberus Hemisphere Enhanced', 'image_url': 'images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'images/valles_marineris_enhanced-full.jpg'}]

In [40]:
# Create dictionary for all info scraped from sources above
mars_dict={
    "news_title":news_title,
    "news_p":news_p,
    "featured_image_url":featured_image_url,
    "fact_table":fact_table,
    "hemisphere_images":hemisphere_image_urls
}

In [41]:
mars_dict

{'news_title': "NASA's Self-Driving Perseverance Mars Rover 'Takes the Wheel'",
 'news_p': 'The agency’s newest rover is trekking across the Martian landscape using a newly enhanced auto-navigation system.',
 'featured_image_url': 'https://spaceimages-mars.com/image/featured/mars2.jpg',
 'fact_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Profile</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n   

In [ ]:
!jupyter nbconvert --to script missions_to_mars.ipynb